In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib import rc
from matplotlib.colors import LinearSegmentedColormap as colormap
from IPython.display import HTML
import math
import os, sys
sys.path.append(os.path.relpath("../src"))
from kitaev_chain import kitaev_chain_model, quench_simulation, U
FFwriter=FFMpegWriter(fps=30, extra_args=['-vcodec', 'libx264'])
from utils import intersections, to_n, s0, sx, sy, sz, maj_ordered, tensor_product, canon_eigen, expm, c, zero, LOSCHMIDT_BDG, LOSCHMIDT_TFIM, STATES

In [3]:
def fermion_chain_from_spin_chain_params_parrallel_field(n_sites, J, h_z, h_edges_x):
    n = n_sites + 2
    mu = np.ones(n) * h_z; mu[0] = mu[-1] = 0
    t = np.ones(n - 1) * J; t[0] = t[-1] = h_edges_x
    t *= .5
    delta = -t
    return kitaev_chain_model(n, mu, t, delta, True)
def fermion_chain_from_spin_chain_params(n, J, h_z, h_edges_x):
    mu = np.ones(n) * h_z
    t = np.ones(n - 1) * J
    t *= .5
    delta = -t
    return kitaev_chain_model(n, mu, t, delta, False)

In [8]:
n = 5
P = np.zeros((2 * n, 2 * n)).astype(complex)
_block = np.ones((2, 2)).astype(complex)
_block[0, 1] = 1j
_block[1, 1] = -1j
for i in range(n):
    P[2 * i: 2 * (i + 1), 2 * i: 2 * (i + 1)] = _block
P = P[list(range(0, 2 * n, 2)) + list(range(1, 2 * n, 2))] * 0.5 ** 0.5
P_dag = P.T.conj()

In [21]:
H0 = fermion_chain_from_spin_chain_params(n, 1, 0, 1).bdg_hamiltonian()
#H0 = fermion_chain_from_spin_chain_params(n, 0, 1, 1).bdg_hamiltonian()
evals_H, evecs_H = np.linalg.eigh(H0)
evals_sorted_H, evecs_sorted_H = canon_eigen(evals_H, evecs_H)

print(np.allclose(evecs_H.conj().T @ H0 @ evecs_H, np.diag(evals_H)))

True


In [ ]:
V1 = np.round(2 ** .5 * (evecs_sorted_H)[:,[0, n]],2)
V2 = np.hstack([V1[:,0:1] + 1j * V1[:,1:2], V1[:,0:1] - 1j * V1[:,1:2]])
np.allclose(-V2[:,1:2], np.vstack([V2[n:, 0:1], V2[:n, 0:1]])),np.allclose(-V1[:,1:2], np.vstack([V1[n:, 0:1], V1[:n, 0:1]]))


True

# We need to find how to always